In [148]:
import requests
import json
import numpy as np
from datetime import datetime

# The following code is a modified query from 'Getting started with The Graph in Python - Intro' 
# available at https://github.com/danielzak/thegraph-intro/blob/main/0.1-Intro.ipynb
# 
# We use polling to create a simple "whale alert" function. It monitors a Uniswap pair for 
# transactions above a certain threhold.
#
# This first cell is the code with a few comments, next cell is an example of a polling function 
# that polls swaps in a pair and executes when a certain condition is met.

query = """query {
    swaps(first:100, pair: "0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc", orderBy: timestamp, orderDirection: desc) {
      timestamp
      amountUSD
    }
}"""

# Call the public, hosted TheGraph endpoint
url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2'
r = requests.post(url, json={'query': query})
#print(r.status_code)

# The request returns a json structured string.
json_data = json.loads(r.text)
#print(json_data)

# Create an empty Numpy array
arr = np.empty((0,2), int)

# Populate the Numpy array, while converting Unix timestamps to datetime objects and keeing the sender hex address as a string
for l in json_data['data']['swaps']:
    arr = np.append(arr, np.array([[datetime.fromtimestamp(int(l['timestamp'])), np.float(l['amountUSD'])]]), axis=0)


In [149]:
# A polling function is made using Polling2. The above code is wrapped as a function that can be called continously.
# This code works inside this Jupyter notebook, but polling should be done in a standalone .py file. 

# Polling2 needs to be installed first using "pip install polling2"
# Polling2 documentation available at https://polling2.readthedocs.io/en/latest/index.html

import polling2
import requests

query = """query {
    swaps(first:100, pair: "0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc", orderBy: timestamp, orderDirection: desc) {
      timestamp
      amountUSD
    }
}"""

# Modify this function to change what happens during each polling cycle, and how to break the cycle
def whale_watch():
    url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2'
    r = requests.post(url, json={'query': query})
    json_data = json.loads(r.text)
    arr = np.empty((0,2), int)
    
    for l in json_data['data']['swaps']:
        arr = np.append(arr, np.array([[datetime.fromtimestamp(int(l['timestamp'])), np.float(l['amountUSD'])]]), axis=0)
    
    # Set the cutoff in USD for the whale alert. Swaps above 100000 are rare, set to 1000-10000 for the condition to trigger
    a = arr[arr[:,1]>100000]
    print(a)

    return a
        
# The polling method
# step=10 sets the time between each poll to ten seconds
# lamba is the break condition for the poll, in this case it breaks when there the list of swaps above the threshold is not empty

polling2.poll(
    lambda: whale_watch().any() != 0,
    step=10,
    poll_forever=True)

[]
[]
[]
[]


KeyboardInterrupt: 